In [169]:
# Importing required libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

import time
import csv
import requests
import re

lank_list = []
with open('lankar.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        lank_list += row
print(lank_list[0:10])
        


['https://www.blocket.se/annons/1001931826', 'https://www.blocket.se/annons/1002069529', 'https://www.blocket.se/annons/1001631939', 'https://www.blocket.se/annons/1001732958', 'https://www.blocket.se/annons/1001732957', 'https://www.blocket.se/annons/1002350904', 'https://www.blocket.se/annons/1002373997', 'https://www.blocket.se/annons/1002200353', 'https://www.blocket.se/annons/1002496767', 'https://www.blocket.se/annons/1002461372']


In [175]:


# Kör utan GUI
options = Options()
options.add_argument("--headless")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")
bildata = []

# # Initialization of variables
# base_url = "https://www.blocket.se"
# pagination_url = "https://www.blocket.se/bilar/sok?filter=%7B%22key%22%3A%22price%22%2C%22range%22%3A%7B%22start%22%3A%2220000%22%2C%22end%22%3A%22%22%7D%7D&filter=%7B%22key%22%3A%22modelYear%22%2C%22range%22%3A%7B%22start%22%3A%222005%22%2C%22end%22%3A%22%22%7D%7D&filter=%7B%22key%22%3A%22ownershipType%22%2C%22values%22%3A%5B%22K%C3%B6pa%22%5D%7D&filter=%7B%22key%22%3A%22fuel%22%2C%22values%22%3A%5B%22El%22%5D%7D&q=EJ+a-traktor&page="
# total_no_of_pages = 2 # Verkligt antal sidor
# # total_no_of_pages = 2 # För test

link_list = []
bil_data = []

# Start Chrome WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

for i in lank_list[3000:10000]:
    driver.get(i)
    time.sleep(2)  # Wait for JS to load
    
    
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    html_str = str(soup)

    make = re.findall(r'"position"\s*:\s*4\s*,\s*"name"\s*:\s*"([^"]+)"', html_str)
    model = re.findall(r'"position"\s*:\s*5\s*,\s*"name"\s*:\s*"([^"]+)"', html_str)
    matches = re.findall(r'sc-hanfos-4 eMJjcF">([^<]+)<', html_str)
    bil_data.append((i, matches, make, model))
    


with open('bil_data3.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['URL','Bränsle', 'Låda', 'Miltal', 'MY', 'Kaross', 'Drift'])  # Rubrikrad
    for bil in bil_data:
        csv_writer.writerow([bil])
# with open('soup.csv', 'w', newline='', encoding='utf-8') as csvfile:
#     csv_writer = csv.writer(csvfile)
#     csv_writer.writerow(soup)  # Rubrikrad


# Stäng WebDriver
driver.quit()


In [325]:
list_pris = []
soup_list = []
price = ""
dragkrok = ""
with open("data_bil_mod.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        list_pris.append(row)

# # Skriv ut listan
# print(list_pris[1][0])
options = Options()
options.add_argument("--headless")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")
bildata = []

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

for index, i in enumerate(list_pris):
    dragkrok = ""
    # print(i[0])
    driver.get(i[0])
    time.sleep(2)  # Wait for JS to load
    
    
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    html_str = str(soup)
    # soup_list.append(html_str)
    
    price = re.findall(r'title"\s*/>\s*<meta\s+content="\s*(\d+\s?\d*\s?kr)', html_str)
    
# Fick inte riktigt till iterationen här, så det blev väldigt dålig kod med if-satser här istället
    try:
        element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[3]/div')
        # text_innehall = element.text.lower()
        text_innehall = element.get_attribute("textContent").lower()
        if "drag" in text_innehall:
            dragkrok = "dragkrok"
        else:
            element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[6]')
            text_innehall2 = element.get_attribute("textContent").lower()
            if "drag" in text_innehall2:
                dragkrok = "dragkrok"
            else:
                element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[2]')
                text_innehall2 = element.get_attribute("textContent").lower()
                if "drag" in text_innehall2:
                    dragkrok = "dragkrok"
                else:
                    element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[3]')
                    text_innehall2 = element.get_attribute("textContent").lower()
                    if "drag" in text_innehall2:
                        dragkrok = "dragkrok"
                    else:
                        element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[4]')
                        text_innehall2 = element.get_attribute("textContent").lower()
                        if "drag" in text_innehall2:
                            dragkrok = "dragkrok"
                        else:
                            element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[5]')
                            text_innehall2 = element.get_attribute("textContent").lower()
                            if "drag" in text_innehall2:
                                dragkrok = "dragkrok"
                            else:
                                element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[6]')
                                text_innehall2 = element.get_attribute("textContent").lower()
                                if "drag" in text_innehall2:
                                    dragkrok = "dragkrok"
                                else:
                                    element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[7]')
                                    text_innehall2 = element.get_attribute("textContent").lower()
                                    if "drag" in text_innehall2:
                                        dragkrok = "dragkrok"
                                    else:
                                        element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[8]')
                                        text_innehall2 = element.get_attribute("textContent").lower()
                                        if "drag" in text_innehall2:
                                            dragkrok = "dragkrok"
                                        else:
                                            element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[9]')
                                            text_innehall2 = element.get_attribute("textContent").lower()
                                            if "drag" in text_innehall2:
                                                dragkrok = "dragkrok"
                                            else:
                                                element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[10]')
                                                text_innehall2 = element.get_attribute("textContent").lower()
                                                if "drag" in text_innehall2:
                                                    dragkrok = "dragkrok"
                                                else:
                                                    element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[11]')
                                                    text_innehall2 = element.get_attribute("textContent").lower()
                                                    if "drag" in text_innehall2:
                                                        dragkrok = "dragkrok"
                                                    else:
                                                        element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[12]')
                                                        text_innehall2 = element.get_attribute("textContent").lower()
                                                        if "drag" in text_innehall2:
                                                            dragkrok = "dragkrok"
                                                        else:
                                                            element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[13]')
                                                            text_innehall2 = element.get_attribute("textContent").lower()
                                                            if "drag" in text_innehall2:
                                                                dragkrok = "dragkrok"
                                                            else:
                                                                element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[14]')
                                                                text_innehall2 = element.get_attribute("textContent").lower()
                                                                if "drag" in text_innehall2:
                                                                    dragkrok = "dragkrok"
                                                                else:
                                                                    element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[15]')
                                                                    text_innehall2 = element.get_attribute("textContent").lower()
                                                                    if "drag" in text_innehall2:
                                                                        dragkrok = "dragkrok"
                                                                    else:
                                                                        element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[16]')
                                                                        text_innehall2 = element.get_attribute("textContent").lower()
                                                                        if "drag" in text_innehall2:
                                                                            dragkrok = "dragkrok"
                                                                        else:
                                                                            element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[17]')
                                                                            text_innehall2 = element.get_attribute("textContent").lower()
                                                                            if "drag" in text_innehall2:
                                                                                dragkrok = "dragkrok"
                                                                            else:
                                                                                element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[18]')
                                                                                text_innehall2 = element.get_attribute("textContent").lower()
                                                                                if "drag" in text_innehall2:
                                                                                    dragkrok = "dragkrok"
                                                                                else:
                                                                                    element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[19]')
                                                                                    text_innehall2 = element.get_attribute("textContent").lower()
                                                                                    if "drag" in text_innehall2:
                                                                                        dragkrok = "dragkrok"
                                                                                    else:
                                                                                        element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[20]')
                                                                                        text_innehall2 = element.get_attribute("textContent").lower()
                                                                                        if "drag" in text_innehall2:
                                                                                            dragkrok = "dragkrok"
                                                                                        else:
                                                                                            element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[21]')
                                                                                            text_innehall2 = element.get_attribute("textContent").lower()
                                                                                            if "drag" in text_innehall2:
                                                                                                dragkrok = "dragkrok"
                                                                                            else:
                                                                                                element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[22]')
                                                                                                text_innehall2 = element.get_attribute("textContent").lower()
                                                                                                if "drag" in text_innehall2:
                                                                                                    dragkrok = "dragkrok"
                                                                                                else:
                                                                                                    element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[23]')
                                                                                                    text_innehall2 = element.get_attribute("textContent").lower()
                                                                                                    if "drag" in text_innehall2:
                                                                                                        dragkrok = "dragkrok"
                                                                                                    else:
                                                                                                        element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[24]')
                                                                                                        text_innehall2 = element.get_attribute("textContent").lower()
                                                                                                        if "drag" in text_innehall2:
                                                                                                            dragkrok = "dragkrok"
                                                                                                        else:
                                                                                                            element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[25]')
                                                                                                            text_innehall2 = element.get_attribute("textContent").lower()
                                                                                                            if "drag" in text_innehall2:
                                                                                                                dragkrok = "dragkrok"
                                                                                                            else:
                                                                                                                element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[26]')
                                                                                                                text_innehall2 = element.get_attribute("textContent").lower()
                                                                                                                if "drag" in text_innehall2:
                                                                                                                    dragkrok = "dragkrok"
                                                                                                                else:
                                                                                                                    element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[27]')
                                                                                                                    text_innehall2 = element.get_attribute("textContent").lower()
                                                                                                                    if "drag" in text_innehall2:
                                                                                                                        dragkrok = "dragkrok"
                                                                                                                    else:
                                                                                                                        element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[28]')
                                                                                                                        text_innehall2 = element.get_attribute("textContent").lower()
                                                                                                                        if "drag" in text_innehall2:
                                                                                                                            dragkrok = "dragkrok"
                                                                                                                        else:
                                                                                                                            element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[29]')
                                                                                                                            text_innehall2 = element.get_attribute("textContent").lower()
                                                                                                                            if "drag" in text_innehall2:
                                                                                                                                dragkrok = "dragkrok"
                                                                                                                            else:
                                                                                                                                element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[30]')
                                                                                                                                text_innehall2 = element.get_attribute("textContent").lower()
                                                                                                                                if "drag" in text_innehall2:
                                                                                                                                    dragkrok = "dragkrok"
                                                                                                                                else:
                                                                                                                                    element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[31]')
                                                                                                                                    text_innehall2 = element.get_attribute("textContent").lower()
                                                                                                                                    if "drag" in text_innehall2:
                                                                                                                                        dragkrok = "dragkrok"
                                                                                                                                    else:
                                                                                                                                        element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[2]/div[2]/div[4]/div[1]/div[2]/ul/li[32]')
                                                                                                                                        text_innehall2 = element.get_attribute("textContent").lower()
                                                                                                                                        if "drag" in text_innehall2:
                                                                                                                                            dragkrok = "dragkrok"
                                                                                                                                        else:
                                                                                                                                            dragkrok = "ej"
            
                                            
                                                
        
    except Exception:
        pass
        
    list_pris[index].append((price[0] if price else None,dragkrok))

    # bil_data.append((i, matches, make, model))
# print(list_pris)

with open('bil_data5.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['URL','Bränsle', 'Låda', 'Miltal', 'MY', 'Kaross', 'Drift'])  # Rubrikrad
    for bil in list_pris:
        csv_writer.writerow([bil])

# with open('bil_data5soup.csv', 'w', newline='', encoding='utf-8') as csvfile:
#     csv_writer = csv.writer(csvfile)
#     csv_writer.writerow([soup_list])

driver.quit()

In [333]:
list_pris2 = []
soup_list2 = []
batteri = ""
bildata2 = []
titel = ""

with open("data_bil2.csv", newline="", encoding="utf-8") as csvfile:
# with open("test.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        list_pris2.append(row)

# # Skriv ut listan
# print(list_pris[1][0])
options = Options()
options.add_argument("--headless")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")


service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

for index, i in enumerate(list_pris2):
    titel = ""
    # print(i[0])
    driver.get(i[0])
    time.sleep(2)  # Wait for JS to load
    
    
    # page_content = driver.page_source
    # soup = BeautifulSoup(page_content, 'html.parser')
    
    # html_str = str(soup)
    # soup_list.append(html_str)
    
    # price = re.findall(r'title"\s*/>\s*<meta\s+content="\s*(\d+\s?\d*\s?kr)', html_str)
    

    try:
        element = driver.find_element("xpath", '//*[@id="skip-tabbar"]/div[2]/div[2]/div[1]/div/article/div[1]/div[2]/div[1]/div[1]/h1')
        # text_innehall = element.text.lower()
        titel = element.get_attribute("textContent").lower()

    except:
        pass
        
    list_pris2[index].append((titel))

    # bil_data.append((i, matches, make, model))
# print(list_pris2)

with open('bil_data6.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['URL','Bränsle', 'Låda', 'Miltal', 'MY', 'Kaross', 'Drift'])  # Rubrikrad
    for bil in list_pris2:
        csv_writer.writerow([bil])

# with open('bil_data5soup.csv', 'w', newline='', encoding='utf-8') as csvfile:
#     csv_writer = csv.writer(csvfile)
#     csv_writer.writerow([soup_list])

driver.quit()